In [32]:
from matplotlib import pyplot
from PIL import Image
import numpy as np
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import cv2
classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
detector = MTCNN()
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')




In [33]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = pyplot.imread(filename)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

In [34]:
def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat


In [35]:
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        print('>face is Match (%.3f <= %.3f)' % (score, thresh))
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))

In [36]:
# create the detector, using default weights
detector = MTCNN()
# detect faces in the image
results = detector.detect_faces('../suspect_images/gallery4.jpg')

InvalidImage: Image not valid.

In [ ]:
Image.open('gallery4.jpg')

In [37]:
# ** From fyp project code

def recognize_face(frame, required_size=(224, 224)):
    detector = MTCNN()
    results = detector.detect_faces(frame)
    # print(results[0])

    # try:
    samples = []
    for faces in results:
        x1, y1, width, height
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = frame[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)

        sample = asarray(face_array, 'float32')
        samples.append(preprocess_input(sample, version=2))

    yhat = []
    for f in samples:
        yhat.append(model.predict(np.expand_dims(f, axis=0)))

    return yhat

In [40]:
def recognize_face(frame, required_size=(224, 224)):
    
    pixels = pyplot.imread(frame)
#     print(pixels)
    results = classifier.detectMultiScale(pixels)
       
#     try:
    samples = []
    for faces in results:
        x1, y1, width, height = faces
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = np.asarray(image)

        sample = np.asarray(face_array, 'float32')
        samples.append(preprocess_input(sample, version=2))

    yhat = []
    for f in samples:
        yhat.append(model.predict(np.expand_dims(f, axis=0)))

    return yhat

In [41]:
import os

In [42]:
f1 = recognize_face(os.path.join('img','gallery4.jpg'))

In [43]:
f2 = recognize_face('cv_image.png')
# len(str(f1[0][0]))

error: OpenCV(4.4.0) /tmp/pip-req-build-vu_aq9yd/opencv/modules/objdetect/src/cascadedetect.cpp:1389: error: (-215:Assertion failed) scaleFactor > 1 && _image.depth() == CV_8U in function 'detectMultiScale'


In [ ]:
is_match(f1,f2)

In [44]:
np.save(os.path.join('img','face.npy'), f1)

In [6]:
a = np.load('face.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../face.npy'

In [ ]:
a

In [ ]:
is_match(a, f1)

In [ ]:
len(a)